In [107]:
import os
import fnmatch

In [108]:
def get_ignore_patterns(start_path):
    patterns = []
    
    # Read all .gitignore files
    for root, dirs, files in os.walk(start_path):
        if '.gitignore' in files:
            with open(os.path.join(root, '.gitignore'), 'r') as f:
                for line in f.readlines():
                    line = line.strip()  # remove whitespace
                    if line and not line.startswith('#'):  # ignore empty lines and comments
                        # Add the root to the pattern to correctly match subdirectories
                        pattern = os.path.join(root, line)
                        patterns.append(pattern)    
    patterns.append('./*.ipynb')
    return patterns

def get_file_structure(start_path, ignore_paths):
    file_structure = {}
    ignore_patterns = get_ignore_patterns(start_path)
    for root, dirs, files in os.walk(start_path, topdown=True):
        dirs[:] = [d for d in dirs if d not in ignore_paths]  # remove the ignored directories from dirs
        files[:] = [f for f in files if f not in ignore_paths]  # remove the ignored directories from dirs        
        for file in files:
            path = os.path.join(root, file)
            if not any(fnmatch.fnmatch(path, pattern) for pattern in ignore_patterns):  # only add if not in ignore list
                current_level = file_structure
                path_parts = path.split(os.sep)[1:]  # split the path into parts and ignore the first part (the start_path)
                for part in path_parts[:-1]:  # for all parts except the last one (the file itself)
                    current_level = current_level.setdefault(part, {})
                current_level[path_parts[-1]] = None  # set the file as a key with value None
    return file_structure

def print_file_structure(structure, descriptions, indent_level=0):
    indent = '  ' * indent_level  # two spaces per indent level
    for name, substructure in structure.items():
        if substructure is None:  # it's a file
            description = descriptions.get(name, '')
            if description == '':
                print("{}- {}".format(indent, name))
            else:
                print("{}- {}: {}".format(indent, name, description))
        else:  # it's a directory
            print(f'{indent}- {name}/')
            print_file_structure(substructure, descriptions, indent_level+2)


In [121]:
description = {
    "main.py" : "a Flask server Script that handles user registration, login, and interactive chat sessions using SocketIO, with a chatbot utilizing GPT for generating responses.",
    "simple_gpt_chatbot.py" : "Interfaces with OpenAI's GPT-3.5 model to generate responses for a chatbot, using the historical data of a chat session retrieved from a database.",
    "model.py" : "Defines the structure of the database tables for a chat application, including Users, ChatSessions, ChatMessages, Chatbots, and the relationships between them.",
    "database.py" : "Initializes an SQLAlchemy database instance for the application to interact with the database.",
    "App.js" : "A React-based client-side application script managing user authentication, registration, chat sessions using Socket.IO, and component rendering according to user authentication status.",
    "Chat.js" : "A React component managing the real-time chat interface, handling message sending/receiving via Socket.IO, maintaining chat session state, and rendering the chat messages and input field." ,
    "Input.js" : "A React component that renders a message input field and send button, handling message submission and maintaining the state of the typed message.",
    "Login.js" : "A React component that renders a login form with input fields for ID and passcode, handles form submission, and performs login request to the server using axios.",
    "Message.js" : "A React component that renders a chat message, including the profile photo, username (for remote messages), and the content of the message.",
    "Registration.js" : "A React component that renders a registration form with input fields for ID and passcode, handles form submission, performs registration request to the server using axios, and subsequently performs login.",
    "SocketContext.js" : "A React context that provides a Socket.IO socket instance to its child components.",
    "index.js" : "The entry point of the React application, rendering the root component (App) into the DOM, enabling React Strict Mode, and optionally measuring performance using the reportWebVitals function.",    
}

In [122]:
start_path = "."
ignore_files = ["venv", "node_modules", "build", ".git", "logo.svg", "public", "Procfile", ".ipynb_checkpoints", "LICENSE", "README.md", "copilot.ipynb", ".gitignore", ".env", ".DS_Store"]
# "package-lock.json", "package.json"
file_structure = get_file_structure(start_path, ignore_files)
print_file_structure(file_structure, description, 0)

- flask/
    - requirements.txt
    - database.py: Initializes an SQLAlchemy database instance for the application to interact with the database.
    - model.py: Defines the structure of the database tables for a chat application, including Users, ChatSessions, ChatMessages, Chatbots, and the relationships between them.
    - main.py: a Flask server Script that handles user registration, login, and interactive chat sessions using SocketIO, with a chatbot utilizing GPT for generating responses.
    - simple_gpt_chatbot.py: Interfaces with OpenAI's GPT-3.5 model to generate responses for a chatbot, using the historical data of a chat session retrieved from a database.
- web_chat/
    - src/
        - reportWebVitals.js
        - App.css
        - index.js: The entry point of the React application, rendering the root component (App) into the DOM, enabling React Strict Mode, and optionally measuring performance using the reportWebVitals function.
        - index.css
        - App.test.js
 